In [ ]:
from libs import *
import string
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

pd.set_option('display.max_colwidth', 50)
plt.style.use(['seaborn-whitegrid', 'seaborn-poster'])

In [ ]:
# Run this line to install the SpaCy English language model.
!python -m spacy download en_core_web_sm

# Data Preprocessing
<a id='contents'></a>
Before we can fit models to our Reddit post text data there are some important steps for us to undertake.

In this exercise our goal is to create a [Bag of Words](https://en.wikipedia.org/wiki/Bag-of-words_model) (BoW) representation of out text. The approach is quite simplistic and doesn't take account of the order that words appear in, only whether they appear in a document or not. Despite this though BoW models have been shown to yield consistently good results.

1. [Initial data analysis](#section1)
2. [Data preprocessing steps](#section2)
3. [Vectorizers](#section3)

First we load some raw post data from a CSV file.

In [ ]:
posts_file = '../datasets/all_reddit_labelled.csv'
df_posts = pd.read_csv(posts_file)
df_posts['created_at'] = pd.to_datetime(df_posts['created_at'])
df_posts = df_posts[~df_posts['label'].isin(['test strategy'])]

<a id='section1'></a>
## 1. Initial data analysis
[back](#contents)

[Initial data analysis](https://en.wikipedia.org/wiki/Data_analysis#Initial_data_analysis) is commonly the first step when starting a data science project. It give us a better understanding of the data and helps to identify any issues with the data that might prevent us from usiung the fdata in our modelling tasks (for example: missing data, inconsistent column values).

First lets look at some of the raw data. By default calling `head()` prints out the first 5 rows of data.

In [ ]:
df_posts.head()

Let's drill down in more detail on the title and body text (and label) from a random sample of 5 posts 

In [ ]:
for i, row in df_posts[~df_posts['label'].isnull()].sample(n=5, random_state=42).iterrows():
    print('-------------------------')
    print(row['title'])
    print()
    print(row['body'])
    print()
    print('Label: ', row['label'])
    print()

Next we will look at the temporal distribution (when were these posts written? How many where written per week?).

In [ ]:
df_raw = pd.read_csv('../datasets/reddit_scrape.csv')
df_raw['created_at'] = pd.to_datetime(df_raw['created_at'])
ax = df_raw.set_index('created_at').resample('W').size().plot(figsize=(10, 5))
ax.set_ylabel('# of posts (weekly)');

Finally, let's look at the labels and their prevalence in the dataset. Most categories appear with approximately similar frequency but the other category is much more frequent. We need to be conscious of this when we fit our models since inbalanced clases like this can lead to models with poor predictive accuracy. There are strategies we can use to address this though.

In [ ]:
ax = df_posts['label'].value_counts().plot(kind='bar', figsize=(10, 5))
ax.set_ylabel('# of posts')

<a id='section2'></a>
## 2. Data preprocessing steps
[back](#contents)

The goal of preprocessing is to get our raw text data into a suitable form that we can use to train a machine learning model.

During this stage, text data is typically cleaned of superfluous 'noise' - for example extra white space, punctuation and very commonly used words (know as 'stop words') can be removed. 

Text is also normalized in order to reduce the size of the overall vocabulary - e.g. by converting text to lower cases so that for example 'Live Conversation' and 'live conversation' are treated in the same way. 

Note that the value of some preprocessing steps is highly dependent on the particular task and the source of the text. Some punctuation for example might be important to the meaning of the text - e.g. emoticons: `:)` `:(` ), but most of the steps we highlight here are commonly used in a range of applications with good results.

Consider the following text.

In [ ]:
raw_text = "I love painting (I've always loved it) so I was running down the road with $10 in my hand to buy some more paints." \
    + " Unfortunately the shop was closed when I arrived!!??!"
print(raw_text)

### Remove punctuation (cleaning)
We can remove all the punctuation from the text. Note that  here we choose to keep $ signs since we think they could be relevant to the task. `PUNCT_TO_REMOVE` is a list of symbol that we intend to remove from our text.

In [ ]:

PUNCT_TO_REMOVE = string.punctuation.replace('$', '')
print(PUNCT_TO_REMOVE)

In [ ]:
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

raw_text_np = remove_punctuation(raw_text)
print(raw_text_np)

### Convert to lower (normalize)
Converting all the text to lower case reduces other overall size of the vocabulary.

In [ ]:
raw_text_np_lower = raw_text_np.lower()
print(raw_text_np_lower)

### Tokenising, removing stopwords and 'lemmatizing' text using Spacy

[Spacy](https://spacy.io/) is a NLP library that can be used to tokenise text and perform many cleaning and normalization tasks. It uses 'language models' that have been trained on a large corpus of text in the target language. The language model can be used to identify stop words and also perform 'lemmatization'. This is a process by which multiple words are mapped to their semantic 'root' For example:

`'loving', 'lover', 'loves', 'loved'`

can all be mapped to the root word 

`'love'`

First we create a model for our chosen language

In [ ]:
nlp = spacy.load("en_core_web_sm")
doc = nlp(raw_text_np_lower)

We can use the model to parse our text and pick out the 'tokens' that it is made up of.

In [ ]:
print("")
print(list([t.text for t in doc]))

We can also use it to identify and therefore remove stop words

In [ ]:
print(list([t.text for t in doc if not t.is_stop]))

Finally we can use the parsed tokens to convert word to their lemmatized root, further normalizing out text

In [ ]:
print(list([t.lemma_ for t in doc if not t.is_stop]))

It does this by first identifying parts of speech (using a pre-training NLP model):

In [ ]:
print(list([t.pos_ for t in doc if not t.is_stop]))

The data is starting to look the way we want it for our model, much of the noise is removed and words have been mapped to thier root form

### Preprocessing pipeline

We can plug these steps together to create a pipline for our reddit post data. We will:
- Join the title and body of the post
- Clean up the white space in the document (e.g. get rid of new lines, tabs etc.)
- Convert to lowercase
- Remove most of the punctation
- Lemmatize the words using Spacy and remove the common 'stop words'

In [ ]:
def lemmatize_and_remove_stop_words(text):
    doc = nlp(text)
    return " ".join([t.lemma_ for t in doc if not t.is_stop])

    
raw_texts = df_posts['title'].fillna(' ') + ' ' + df_posts['body'].fillna(' ')

print('** Before processing: **')
print(raw_texts[0])
print()


raw_texts = raw_texts.str.replace('[^\w\' ]', '')
raw_texts = raw_texts.str.lower()
raw_texts = raw_texts.map(remove_punctuation)

print('** After 1st stage of processing: **')
print(raw_texts[0])
print()

raw_texts = raw_texts.map(lemmatize_and_remove_stop_words)

print('** After 2nd stage of processing: **')
print(raw_texts[0])

<a id='section3'></a>
## 3. Vectorizers
[back](#contents)

Once we have cleaned and normalized our text data, we need to convert it in to a form that machine learning models can uderstand - i.e. a bag of words. We do this using a vectorizer which maps each piece of text to an array of 1s and 0s indictating whether each word in our vocabulary appears in our text or not.

<img src="figures/count_vectorizer.png" width=500>

In the image above, we can see how 4 different country names would be represented by a [count vectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) (which simply counts the number of occurences of each word). The length of the array is the length of our vocabulary which is the number of unique words that appear in all our text documents. If this number is too large, we typically throw away the tokens that occur least frequently throughout the corpus.

In the example below, we can see how each line in a very simple corpus of text can be represented as an array of 1s and 0s.

In [ ]:
cv = CountVectorizer()

texts = [
    'she goes to the zoo',
    'he goes to bed',
    'they like icecream and they like pizza',
    'we go home and then go to bed'
]

bows = cv.fit_transform(texts)
pd.DataFrame(bows.toarray(), columns=cv.get_feature_names())

Vectorizers can also be used to encode combinations of words that appear sequentially (n-grams) for example we might want to count the occurences of all pairs of words in our text in addition to single words.

In [ ]:
cv2gram = CountVectorizer(ngram_range=(1,2))

texts = [
    'she goes to the zoo',
    'he goes to the zoo',
]

bows_2gram = cv2gram.fit_transform(texts)
pd.DataFrame(bows_2gram.toarray(), columns=cv2gram.get_feature_names())

Another type of vectorizer that is commonly used is a [Term Frequency - Inverse Document Frequency vectorizer](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) (tf-idf). This accounts for the frequency of each word within a specific document relative to it's frequency across the whole corpus.

In [ ]:
tfidfv = TfidfVectorizer()

texts = [
    'she goes to the zoo',
    'he goes to bed',
    'they like icecream and they like pizza',
    'we go home and then go to bed'
]

tfidf_bows = tfidfv.fit_transform(texts)
pd.DataFrame(np.round(tfidf_bows.toarray(), 2), columns=tfidfv.get_feature_names())

That's it for preprocessing, we now have our text in a form that our machine learning models can understand and learn from.